In [18]:
import os
import pandas as pd
import numpy as np
import cv2
import rasterio
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchvision.models import resnet50
import csv

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [19]:
data_dir = "G:\\Yehmh\\DNDF\\202309_DNDF\\species\\5m_5m"
model_path = 'G:/Yehmh/_model/0411_DNDF_resnet_74.pth'

X = []  # Features
y = []  # Labels


for folder in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, folder)):  # Check if it's a directory
        species = folder  # Assuming folder name is the species label
        y.append(species)

In [20]:
# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(label_encoder.classes_)

['Acac_au' 'Bisc_ja' 'Brou_pa' 'Call_fo' 'Cinn_bu' 'Cinn_ca' 'Elae_sy'
 'Frax_gr' 'Koel_el' 'Liqu_fo' 'Luec_le' 'Mach_zu' 'Magn_co' 'Pter_in'
 'Quer_gl' 'Zelk_se' '_not_tree']


In [21]:
# Define transformations for the images
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

In [22]:
class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.resnet = resnet50(pretrained=True)  # Load pretrained ResNet-50
        # Replace the last fully connected layer with a new one
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Function to extract coordinates from TIF files
def extract_coordinates(tif_file):
    with rasterio.open(tif_file) as src:
        crs = src.crs
        bounds = src.bounds
        center_x = (bounds.left + bounds.right) / 2
        center_y = (bounds.top + bounds.bottom) / 2
        return crs, (center_x, center_y)

def classify_and_write(unknown_photos_dir, output_csv_path):

    # Initialize lists to store unknown photo paths, coordinates, and predictions
    unknown_photos = []
    unknown_coordinates = []

    # Iterate over unknown TIF files
    for filename in os.listdir(unknown_photos_dir):
        if filename.endswith('.tif'):
            tif_file = os.path.join(unknown_photos_dir, filename)
            
            # Extract coordinates
            crs, coordinates = extract_coordinates(tif_file)
            
            # Append to the list
            unknown_photos.append(tif_file)
            unknown_coordinates.append((filename, crs, coordinates))

    # Now you have a list of unknown photo paths (unknown_photos) and corresponding coordinates (unknown_coordinates)

    # Define a function to classify with threshold
    def classify_with_threshold(probabilities, threshold):
        max_prob, max_index = torch.max(probabilities, dim=1)
        if max_prob.item() < threshold:
            return "unknown"
        else:
            predicted_label = label_encoder.classes_[max_index.item()]
            return predicted_label  # Return the index of the class with the maximum probability


    # Load the trained model
    model = ResNet50(num_classes=len(label_encoder.classes_))
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode

    # Make predictions on the unknown photos
    predictions = []
    for photo_path in unknown_photos:
        # Load and preprocess the image
        image = cv2.imread(photo_path)
        image = cv2.resize(image, (64, 64))  # Resize image if necessary
        image = transform(image)
        image = image.unsqueeze(0)  # Add batch dimension
        
        # Perform inference
        with torch.no_grad():
            output = model(image)
            probabilities = nn.functional.softmax(output, dim=1)  # Apply softmax to get probabilities
            classification = classify_with_threshold(probabilities, threshold=0.7)  # Adjust threshold as needed
            predictions.append(classification)

    # Now you have the predictions for each unknown photo in the list 'predictions'
    # You can proceed to visualize the results on a map using the coordinates
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Filename', 'Latitude', 'Longitude', 'Predicted Species'])  # Write header
        for coord, label in zip(unknown_coordinates, predictions):
            filename, crs, (latitude, longitude) = coord
            writer.writerow([filename, latitude, longitude, label])

In [23]:
# list = [69, 70, 71, 75, 76, 78, 79, 82]
# # list = [76, 78, 79, 82]
# # list = [78]

# for i in list:
#     unknown_photos_dir = f'D:\\Yehmh\\test_py\\202301\\P000{i}\\5m_5m'
#     output_csv_path = f'D:/Yehmh/test_py/202301/P000{i}_resnet_prob_2.csv'
    
#     classify_and_write(unknown_photos_dir, output_csv_path)
    
#     print(i, " done")

In [24]:
folder_paths = []
for i in range(1, 10):
    folder_paths.append(f"G:\\Yehmh\\DNDF\\202309_DNDF\\pix4d file\\00{i}\\5m_5m")
for i in range(10, 16):
    folder_paths.append(f"G:\\Yehmh\\DNDF\\202309_DNDF\\pix4d file\\0{i}\\5m_5m")

i = 1
for folder_path in folder_paths:
    unknown_folder_path = folder_path
    output_csv_path = f'G:\\Yehmh\\DNDF\\202309_DNDF\\results\\{i}_0411_resnet.csv'
    i = i + 1

    classify_and_write(unknown_folder_path, output_csv_path)
    print(folder_path, " done")

c:\Users\CIRES\anaconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\CIRES\anaconda3\envs\test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


G:\Yehmh\DNDF\202309_DNDF\pix4d file\001\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\002\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\003\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\004\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\005\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\006\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\007\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\008\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\009\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\010\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\011\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\012\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\013\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\014\5m_5m  done
G:\Yehmh\DNDF\202309_DNDF\pix4d file\015\5m_5m  done
